<a href="https://colab.research.google.com/github/alwiyahya99/MachineLearning/blob/main/2_Membuat_Model_untuk_Klasifikasi_Dua_Kelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Membuat Dan Melatih Model NN Dengan Tensorflow Dan Keras : Membuat Model untuk Klasifikasi Dua Kelas**

## **1. IMPORT LIBRARY DAN PANGGIL DATASET**

In [3]:
import pandas as pd
df = pd.read_csv('citrus.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


In [7]:
df.head(10000)

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


## **2. MEMPROSES DATASET**

mengubah label menjadi numerik, karena jaringan saraf tiruan tidak bisa memproses data string

In [8]:
df.name[df.name == 'orange'] = 0
df.name[df.name == 'grapefruit'] = 1

<ipython-input-8-e9c3e7476411>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name[df.name == 'orange'] = 0
<ipython-input-8-e9c3e7476411>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name[df.name == 'grapefruit'] = 1


Mengubah dataframe menjadi dataset array

In [10]:
dataset = df.values
dataset

array([[0, 2.96, 86.76, 172, 85, 2],
       [0, 3.91, 88.05, 166, 78, 3],
       [0, 4.42, 95.17, 156, 81, 2],
       ...,
       [1, 15.59, 256.5, 168, 82, 20],
       [1, 15.92, 260.14, 142, 72, 11],
       [1, 16.45, 261.51, 152, 74, 2]], dtype=object)

Memisahkan antara atribut dan label pada dataset.

Dari eksplorasi data kita sebelumnya, dapat diketahui pada 5 kolom terakhir adalah kolom atribut

In [11]:
# pilih 5 kolom terakhir sebagai berikut
X = dataset[:,1:6]
# bilangan sebelum koma untuk memilih baris pada dataframe/dataset
# bilangan setelah koma untuk memilih kolom pada dataframe/dataset

Memisahkan label dari dataset. pada dataset ini label terdapat pada kolom pertama.

In [12]:
y = dataset[:,0]

Melakukan normalisasi pada dataset, agar jaringan saraf tiruan bisa mempelajarinya. menggunakan fungsi fit_transform() dari sebuah objek minMaxScaler dari library SKLearn untuk normalisasi data.

In [15]:
# Normalization
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.        , 0.        , 0.74025974, 0.63529412, 0.        ],
       [0.07042254, 0.00738197, 0.66233766, 0.55294118, 0.01851852],
       [0.10822832, 0.04812589, 0.53246753, 0.58823529, 0.        ],
       ...,
       [0.93624907, 0.97133047, 0.68831169, 0.6       , 0.33333333],
       [0.96071164, 0.99216023, 0.35064935, 0.48235294, 0.16666667],
       [1.        , 1.        , 0.48051948, 0.50588235, 0.        ]])

Memisahkan data LATIH dan data UJI kedalam training dan test set dengan train_test_split. kita gunakan 30% dari sluruh data sebagai uji.

In [16]:
# pisahkan data training dan testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, y, test_size=0.3)

sebelumnya kita mengubah nilai-nilai dataset menjadi bilangan numerik, yang artinya kita mengubah label menjadi data boolean. maka kita perlu mengubah tipe data tersebut menjadi float32.

In [17]:
import numpy as np

Y_train = Y_train.astype(np.float32)
Y_test = Y_test.astype(np.float32)

## **3. MEMBUAT JARINGAN SARAF TIRUAN**

In [20]:
from keras.models import Sequential
from keras.layers import Dense

model yang kita kembangkan adalah :

*   Model Sequential
*   3 buah layer

*   2 Activation Function pada 2 layer pertama : relu
*   layer terakhir isi parameter unit dengan 1 dimana output dari JST kita merupakan satu buah bilangan numerik. Activation Function isi dengan sigmoid karena memetakan probabilitas dari 0 sampai 1. sigmoid cocok memecahkan masalah klasifikasi biner.





In [21]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(5,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

## **4. MENENTUKAN OPTIMIZER DAN LOSS**

kita perlu menentukan optimizer dan loss function dari model kita.
- optimizer kita akan menggunakan stochastic gradient descent (sgd) yang merupakan optimizer yang sangat umum dan cocok dipakai pada dataset yang berukuran kecil.
- loss yang sesuai adalah ‘binary_crossentropy’ karena masalah pada latihan kita kali ini adalah masalah 2 kelas (biner).
- menampilkan akurasi pada setiap proses pelatihan model.

In [26]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])


## **5. MELATIH MODEL**


In [27]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.1729 - accuracy: 0.9290
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1728 - accuracy: 0.9286
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1728 - accuracy: 0.9299
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1724 - accuracy: 0.9281
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1728 - accuracy: 0.9273
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1722 - accuracy: 0.9283
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1724 - accuracy: 0.9304
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1730 - accuracy: 0.9293
Epoch 9/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1728 - accuracy: 0.9279
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 0.1724 - accura

In [28]:
model.evaluate(X_test, Y_test)
# elemen pertama alah loss dan elemen kedua adalah accuracy

94/94 [==============================] - 0s 2ms/step - loss: 0.1871 - accuracy: 0.9263


[0.187113419175148, 0.9263333082199097]